In [ ]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"
import pyrender
import cv2
import glob
import json
import time
import trimesh
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, precision=3)
from scipy.spatial.transform import Rotation as R
from bpc.inference.utils.camera_utils import load_camera_params
from bpc.inference.process_pose import PoseEstimator, PoseEstimatorParams
from bpc.utils.data_utils import Capture, render_mask
import bpc.utils.data_utils as du
import importlib

: 

In [ ]:
def render_mask_cv(obj, K, RT, imsize, mesh_poses):
    h, w = imsize[1], imsize[0]
    mask = np.zeros((h, w), dtype=np.uint8)
    # print("\n \n Bahey \n \n ", len(mesh_poses), mesh_poses[0].shape)
    vertices = obj.vertices
    faces = obj.faces
 
    for pose in mesh_poses:
        # Full transformation: Object -> world -> camera
        verts_homo = np.hstack((vertices, np.ones((vertices.shape[0], 1))))  # Nx4
        world_vertices = (pose @ verts_homo.T).T  # Object to world
        camera_vertices = (RT @ np.hstack((world_vertices[:, :3], np.ones((world_vertices.shape[0], 1)))).T).T[:, :3]  # World to camera
 
        # Now project camera space points
        projected_pts, _ = cv2.projectPoints(
            camera_vertices,
            np.zeros(3),  # Already in camera space
            np.zeros(3),
            K, None
        )
        projected_pts = projected_pts.squeeze().astype(np.int32)
 
        # Fill each triangle face
        for face in faces:
            pts = projected_pts[face]
            cv2.fillConvexPoly(mask, pts, 255)
 
    return mask, None

: 

In [ ]:
# scene_dir = "./datasets/ipd_bop_data_jan25_1/train_pbr/000000/"
# models_dir = './datasets/ipd_bop_data_jan25_1/models_eval/'
scene_dir = "./datasets/ipd_val/val/000001/"
models_dir = './datasets/ipd_models/models/'
cam_ids = ["cam1", "cam2", "cam3"]
image_id = 0
obj_id = 14
obj_id_path = str(1000000+obj_id)[1:]
ply_file = os.path.join(models_dir, f"obj_{obj_id_path}.ply")
obj = trimesh.load(ply_file)
yolo_model_path = f'models/detection/obj_{obj_id}/yolo11-detection-obj_{obj_id}.pt'
pose_model_path = f'models/rot_models/rot_{obj_id}.pth'
 
pose_params = PoseEstimatorParams(yolo_model_path=yolo_model_path,
                                  pose_model_path=pose_model_path,
                                  yolo_conf_thresh=0.01)
pose_estimator = PoseEstimator(pose_params)
t = time.time()
capture = Capture.from_dir(scene_dir, cam_ids, image_id, obj_id)
detections = pose_estimator._detect(capture)
pose_predictions = pose_estimator._match(capture, detections)
pose_estimator._estimate_rotation(pose_predictions)
print(time.time() - t)

for idx in range(len(capture.Ks)):
    plt.figure(figsize=(15, 15))
    plt.imshow(capture.images[idx])
    mask, _ = render_mask_cv(
        obj,
        capture.Ks[idx],
        capture.RTs[idx],
        capture.images[0].shape[:2][::-1],
        [x.pose for x in pose_predictions]
    )
    plt.imshow(mask, alpha=0.5, cmap='jet')
    plt.show()


: 

In [ ]:
# from bpc.pose.models.metrics import compute_add, compute_mvd, compute_rotation_translation_error
 
# all_add, all_mvd, tp, fp, fn = [], [], 0, 0, 0

# for cam_idx in range(len(capture.Ks)):  # Per camera
#     gt_R, gt_t = capture.RTs[cam_idx][:3, :3], capture.RTs[cam_idx][:3, 3:]
 
#     try:
#         pred = pose_predictions[cam_idx]
#     except IndexError:
#         fn += 1
#         continue
 
#     pred_R, pred_t = pred.pose[:3, :3], pred.pose[:3, 3:]
#     # print(pred_R, pred_t) 
#     # Metrics
#     add = compute_add(gt_R, gt_t, pred_R, pred_t, obj.vertices)
#     mvd = compute_mvd(gt_R, gt_t, pred_R, pred_t, obj.vertices)
#     rot_err, trans_err = compute_rotation_translation_error(gt_R, gt_t, pred_R, pred_t)
#     print("\n Rotation error", rot_err, "\n Translation error", trans_err)

 
#     all_add.append(add)
#     all_mvd.append(mvd)
 
#     if trans_err < 10 and rot_err < 10:
#         tp += 1
#     else:
#         fp += 1
 
 
# # Final aggregate results
# recall = tp / (tp + 1e-6)
# precision = tp / (tp + fp + 1e-6)
 
# print(f"\n[Scene {scene_dir}, Image {image_id}]")
# print(f"Precision @ 10mm/10°: {precision:.4f}")
# print(f"Recall @ 10mm/10°: {recall:.4f}")
# print(f"Mean ADD: {np.mean(all_add):.4f} mm")
# print(f"Mean MVD: {np.mean(all_mvd):.4f} mm")

def pairwise_rotation_errors(pred_poses, gt_poses):
    N, M = pred_poses.shape[0], gt_poses.shape[0]
    pred_R = pred_poses[:, :3, :3]  # (N, 3, 3)
    gt_R   = gt_poses[:, :3, :3]    # (M, 3, 3)

    rot_errors = np.zeros((N, M))

    for i in range(N):
        for j in range(M):
            R_rel = pred_R[i].T @ gt_R[j]
            trace = np.trace(R_rel)
            trace = np.clip(trace, -1.0, 3.0)  # numerical stability
            angle_rad = np.arccos((trace - 1) / 2.0)
            rot_errors[i, j] = np.degrees(angle_rad)

    return rot_errors  # shape: (N, M)

def pairwise_translation_errors(pred_poses, gt_poses):
    # Extract translation vectors
    pred_t = pred_poses[:, :3, 3]  # (N, 3)
    gt_t   = gt_poses[:, :3, 3]    # (M, 3)

    # Compute all pairwise L2 distances
    # Shape: (N, M)
    dists = np.linalg.norm(pred_t[:, None, :] - gt_t[None, :, :], axis=-1)
    
    return dists
    

def compute_pose_errors(pred_poses, gt_poses):
    assert pred_poses.shape[0] == gt_poses.shape[0], "Mismatch in number of poses"

    rot_errors = []
    trans_errors = []

    for pred, gt in zip(pred_poses, gt_poses):
        R_pred, t_pred = pred[:3, :3], pred[:3, 3]
        R_gt,   t_gt   = gt[:3, :3],   gt[:3, 3]

        # Translation error
        t_error = np.linalg.norm(t_pred - t_gt)

        # Rotation error (in degrees)
        R_rel = R_pred.T @ R_gt
        trace = np.trace(R_rel)
        angle_rad = np.arccos(np.clip((trace - 1) / 2.0, -1.0, 1.0))
        r_error = np.degrees(angle_rad)

        trans_errors.append(t_error)
        rot_errors.append(r_error)

    return np.array(rot_errors), np.array(trans_errors)


from bpc.pose.models.metrics import compute_add, compute_mvd, compute_rotation_translation_error
 
all_add, all_mvd, tp, fp, fn = [], [], 0, 0, 0

for cam_idx in range(len(capture.Ks)):  # Per camera
    pose_matrix = capture.RTs[cam_idx]
    gt_poses = capture.gt_poses

    poses_world = []
    for pose_pred in pose_predictions:
        poses_world.append(pose_pred.pose)

    poses_world = np.array(poses_world)
        
    poses_camera = pose_matrix @ poses_world

    all_trans_errors = pairwise_translation_errors(poses_camera, gt_poses)
    all_rots_errors = pairwise_rotation_errors(poses_camera, gt_poses)
    best_pred_indices = np.argmin(all_trans_errors, axis=0)  # (M,)
    matched_pred_poses = poses_camera[best_pred_indices]  # shape: (M, 4, 4)

    rot_errors, trans_errors = compute_pose_errors(matched_pred_poses, gt_poses)
    # print(best_pred_indices, matched_pred_poses.shape)
    # print(all_trans_errors)
    # print("\n \n")
    print(rot_errors, trans_errors)
    # print(all_rots_errors)
    break
    # cam_R_w2c, cam_t_w2c, cam_K = capture.RTs[cam_idx][:3, :3], capture.RTs[cam_idx][:3, 3:], capture.Ks[0]
    # print("\n GT Poses \n", pose_matrix.shape, gt_pose.shape, poses_camera.shape)
    # T_cam_w2c = np.eye(4)
    # T_cam_w2c[:3, :3] = cam_R_w2c
    # T_cam_w2c[:3, 3] = cam_t_w2c
    # print(cam_R.shape,"\n")
    # print(cam_t.shape,"\n")
    # print(cam_K.shape,"\n")
 
#     pred_R, pred_t = pred.pose[:3, :3], pred.pose[:3, 3:]
#     # print(pred_R, pred_t) 
#     # Metrics
#     add = compute_add(gt_R, gt_t, pred_R, pred_t, obj.vertices)
#     mvd = compute_mvd(gt_R, gt_t, pred_R, pred_t, obj.vertices)
#     rot_err, trans_err = compute_rotation_translation_error(gt_R, gt_t, pred_R, pred_t)
#     print("\n Rotation error", rot_err, "\n Translation error", trans_err)

 
#     all_add.append(add)
#     all_mvd.append(mvd)
 
#     if trans_err < 10 and rot_err < 10:
#         tp += 1
#     else:
#         fp += 1
 
 
# # Final aggregate results
# recall = tp / (tp + 1e-6)
# precision = tp / (tp + fp + 1e-6)
 
# print(f"\n[Scene {scene_dir}, Image {image_id}]")
# print(f"Precision @ 10mm/10°: {precision:.4f}")
# print(f"Recall @ 10mm/10°: {recall:.4f}")
# print(f"Mean ADD: {np.mean(all_add):.4f} mm")
# print(f"Mean MVD: {np.mean(all_mvd):.4f} mm

: 

In [ ]:
import os
import json

def orthogonalize_rotation(R):
    U, _, Vt = np.linalg.svd(R)
    R_ortho = U @ Vt
    # Ensure proper rotation (det = +1)
    if np.linalg.det(R_ortho) < 0:
        R_ortho *= -1
    return R_ortho

scene_base_dir = "/l/users/ahmed.aly/ipd/val/"
models_dir = "/l/users/ahmed.aly/models/"
output_dir = "./outputs_14_2/"
cam_ids = ["cam1", "cam2", "cam3"]
obj_ids = [14]

scenes_list = sorted(os.listdir(scene_base_dir))
for scene_id, scene in enumerate(scenes_list):
    os.makedirs(os.path.join(output_dir, scene), exist_ok=True)
    scene_dir = os.path.join(scene_base_dir, scene)
    cam_dir = os.path.join(scene_dir, "rgb_cam1")
    images_list = sorted(os.listdir(cam_dir))
    for image_id, image_name in enumerate(images_list):
        for obj_id in obj_ids:    
            obj_id_path = str(1000000+obj_id)[1:]
            ply_file = os.path.join(models_dir, f"obj_{obj_id_path}.ply")
            obj = trimesh.load(ply_file)
            yolo_model_path = f'models/detection/obj_{obj_id}/yolo11-detection-obj_{obj_id}.pt'
            pose_model_path = f'models/rot_models/rot_{obj_id}.pth'

            pose_params = PoseEstimatorParams(yolo_model_path=yolo_model_path,
                                              pose_model_path=pose_model_path,
                                              yolo_conf_thresh=0.01)
            pose_estimator = PoseEstimator(pose_params)
            t = time.time()
            capture = Capture.from_dir(scene_dir, cam_ids, image_id, obj_id)
            detections = pose_estimator._detect(capture)
            pose_predictions = pose_estimator._match(capture, detections)
            pose_estimator._estimate_rotation(pose_predictions)
            print(time.time() - t)

            for cam_idx in range(len(capture.Ks)):  # Per camera
                ## Create cam directory
                os.makedirs(os.path.join(output_dir, scene, "cam"+str(cam_idx+1)), exist_ok=True)
                ## Create image directory inside cam directory
                os.makedirs(os.path.join(output_dir, scene, "cam"+str(cam_idx+1), image_name[:-4]), exist_ok=True)
                os.makedirs(os.path.join(output_dir, scene, "cam"+str(cam_idx+1), image_name[:-4], "sam6d_results"), exist_ok=True)

                
                pose_matrix = capture.RTs[cam_idx]
                print("\n \n", cam_idx, "\n \n", pose_matrix)
                poses_world = np.array([pose_pred.pose for pose_pred in pose_predictions])
                if len(poses_world) > 0:
                    poses_camera = pose_matrix @ poses_world
                    pred_t = poses_camera[:, :3, 3]  # (N, 3)
                    pred_R = poses_camera[:, :3, :3]  # (N, 3, 3)
                    json_preds = []
                    for pred_pose_id in range(len(poses_world)):  
                        pred_R[pred_pose_id] = orthogonalize_rotation(pred_R[pred_pose_id])
                        json_dict = {"scene_id": scene_id,
                                     "image_id": image_id,
                                     "category_id": obj_id,
                                     "R": pred_R[pred_pose_id].tolist(),
                                     "t": pred_t[pred_pose_id].tolist()
                                    }
                        json_preds.append(json_dict)

                    with open(os.path.join(output_dir, scene, "cam"+str(cam_idx+1), image_name[:-4], "sam6d_results/detection_pem.json"), "w") as f:
                        json.dump(json_preds, f, indent=4)


# for cam in cam_ids:

# for idx in range(len(capture.Ks)):
#     plt.figure(figsize=(15, 15))
#     plt.imshow(capture.images[idx])
#     mask, _ = render_mask_cv(
#         obj,
#         capture.Ks[idx],
#         capture.RTs[idx],
#         capture.images[0].shape[:2][::-1],
#         [x.pose for x in pose_predictions]
#     )
#     plt.imshow(mask, alpha=0.5, cmap='jet')
#     plt.show()




# for cam_idx in range(len(capture.Ks)):  # Per camera
#     pose_matrix = capture.RTs[cam_idx]
#     gt_poses = capture.gt_poses

#     poses_world = []
#     for pose_pred in pose_predictions:
#         poses_world.append(pose_pred.pose)

#     poses_world = np.array(poses_world)
        
#     poses_camera = pose_matrix @ poses_world
#     pred_t = poses_camera[:, :3, 3]  # (N, 3)
#     pred_R = poses_camera[:, :3, :3]  # (N, 3, 3)


: 